In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from itertools import combinations
from sklearn.cluster import KMeans
from google.colab import files
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor

# 1. CARREGAR O GRAFO
uploaded = files.upload()
graph_file = next(iter(uploaded))
G = nx.read_graphml(graph_file)
G = nx.relabel_nodes(G, {n: eval(n) for n in G.nodes()})

# 2. PARÂMETROS AVANÇADOS
num_roteadores = int(input("Digite o número de roteadores: "))
rssi_threshold = -70
tx_power = 23  # dBm
freq_mhz = 2400  # MHz

# 3. PRÉ-CÁLCULO DE FSPL
def calc_fspl(distance_m):
    if distance_m == 0:
        return 0
    distance_km = distance_m / 1000.0
    return 20 * np.log10(distance_km) + 20 * np.log10(freq_mhz) + 32.44

# 4. MEMOIZAÇÃO DE CAMINHOS E PERDAS
@lru_cache(maxsize=None)
def get_path_and_loss(source, target):
    try:
        path = nx.shortest_path(G, source=source, target=target, weight='weight')
        obstacle_loss = sum(G[p][q]['weight'] for p, q in zip(path[:-1], path[1:]))
        return path, obstacle_loss
    except (nx.NetworkXNoPath, KeyError):
        return None, float('inf')

# 5. FUNÇÃO DE AVALIAÇÃO PARALELA
def compute_rssi_for_node(node, routers):
    best_rssi = -100.0
    for router in routers:
        path, obstacle_loss = get_path_and_loss(node, router)
        if path is None:
            continue
        euclidean_dist = np.hypot(node[0] - router[0], node[1] - router[1])
        fspl = calc_fspl(euclidean_dist * 0.5)
        rssi = tx_power - fspl - obstacle_loss
        if rssi > best_rssi:
            best_rssi = rssi
    return best_rssi

def evaluate_coverage(G, routers):
    node_list = list(G.nodes())

    with ThreadPoolExecutor() as executor:
        rssi_values = list(executor.map(lambda node: compute_rssi_for_node(node, routers), node_list))

    rssi_values = np.array(rssi_values)
    coverage = np.sum(rssi_values >= rssi_threshold) / len(rssi_values) * 100
    avg_rssi = np.mean(rssi_values[rssi_values > -100])
    return coverage, avg_rssi, rssi_values

# 6. Penalização por proximidade entre roteadores
def router_distance_penalty(routers):
    total = 0
    for a, b in combinations(routers, 2):
        d = np.hypot(a[0] - b[0], a[1] - b[1])
        total += 1 / (d + 1e-3)
    return total

# 7. Busca com clusters e mutação periódica - OTIMIZADA
def find_best_routers(G, num_roteadores, n_iter=50):
    nodes = list(G.nodes())
    best = {'coverage': 0, 'avg_rssi': -100, 'routers': None}
    best_score = -np.inf

    # Centralidade dos nós (grau)
    centrality = nx.degree_centrality(G)
    top_central_nodes = sorted(centrality, key=centrality.get, reverse=True)[:len(nodes)//2]

    # Avaliação inicial com todos os nós para detectar regiões fracas
    _, _, initial_rssi_values = evaluate_coverage(G, [])
    weak_nodes = [node for node, rssi in zip(nodes, initial_rssi_values) if rssi < rssi_threshold]

    # Interseção: nós que são centrais E mal cobertos
    candidate_nodes = list(set(top_central_nodes) | set(weak_nodes))

    for iteration in range(n_iter):
        if len(candidate_nodes) < num_roteadores:
            candidate_nodes = nodes.copy()

        # Mutação a cada 10 iterações
        if iteration % 10 == 0:
            random_idxs = np.random.choice(len(nodes), min(20, len(nodes)), replace=False)
            mutation_nodes = [nodes[i] for i in random_idxs]
            candidate_nodes = list(set(candidate_nodes + mutation_nodes))


        selected_indices = np.random.choice(len(candidate_nodes), size=num_roteadores, replace=False)
        combo = [candidate_nodes[i] for i in selected_indices]

        coverage, avg_rssi, _ = evaluate_coverage(G, combo)
        penalty = router_distance_penalty(combo)
        score = coverage - 0.5 * penalty

        if score > best_score:
            best = {'coverage': coverage, 'avg_rssi': avg_rssi, 'routers': combo}
            best_score = score

            # Atualiza candidatos com base nos vizinhos dos melhores
            new_candidates = set()
            for router in combo:
                neighbors = list(G.neighbors(router)) + [router]
                new_candidates.update(neighbors)
            candidate_nodes = list(new_candidates)

    return best['routers'], best['coverage'], best['avg_rssi']

# 8. EXECUÇÃO
print("\n=== OTIMIZAÇÃO DE ROTEADORES ===")
print(f"Configuração: {num_roteadores} roteadores, {len(G.nodes())} nós")
print(f"TX Power: {tx_power} dBm, Frequência: {freq_mhz/1000} GHz\n")

best_routers, coverage, avg_rssi = find_best_routers(G, num_roteadores)
print("\n=== RESULTADOS ===")
print(f"Melhores posições: {best_routers}")
print(f"Cobertura: {coverage:.1f}% (acima de {rssi_threshold} dBm)")
print(f"RSSI médio: {avg_rssi:.1f} dBm")

# 9. VISUALIZAÇÃO
def plot_simulation(G, routers):
    pos = {n: (n[0], n[1]) for n in G.nodes()}
    plt.figure(figsize=(14, 10))

    _, _, rssi_values = evaluate_coverage(G, routers)

    weight_colors = {
        1: 'gray',
        4: 'green',
        5: 'yellow',
        8: 'red',
        10: 'blue'
    }

    edge_colors = []
    for u, v in G.edges():
        weight = G[u][v].get('weight', 1)
        edge_colors.append(weight_colors.get(weight, 'black'))

    nx.draw_networkx_edges(G, pos, edge_color=edge_colors, width=1.2, alpha=0.6)

    nodes = nx.draw_networkx_nodes(
        G, pos, node_color=rssi_values,
        cmap='RdYlGn', vmin=-90, vmax=-30,
        node_size=80
    )

    nx.draw_networkx_nodes(
        G, pos, nodelist=routers,
        node_color='black', node_size=300,
        edgecolors='yellow', linewidths=2
    )

    plt.colorbar(nodes, label='RSSI (dBm)')
    plt.title(f"Cobertura: {coverage:.1f}% | RSSI Médio: {avg_rssi:.1f} dBm")
    plt.axis('equal')
    plt.axis('off')
    plt.show()

plot_simulation(G, best_routers)
